# Math  1376: Programming for Data Science
---

In [ ]:
import numpy as np #We will use numpy in this lecture
import matplotlib.pyplot as plt
%matplotlib widget

## Module 04: Some useful applications of Modules 01-03
---

In this module, we will now pull together material across our first three modules to solve some practical problems. 

You may find it useful to review the notebooks in those modules beforehand or just simply open some of the notebooks to have their contents available to review as necessary. 

While there are a seemingly endless number of practical problems we can attempt to solve with what we have learned so far, we will focus on three ubiquitous problems in the computational sciences.

- Root-finding. (The content of Part (a) of this module.)

- Numerical integration. (The content of Part (b) of this module.)

- Optimization. (This content is pursued in the homework of this module.)

### A note about calculus concepts and interactive visualizations (i.e., widgets)
---

These topics are commonly studied as applications of calculus concepts. 
However, while we may make passing reference to certain calculus concepts, you do *not* need to know calculus to follow the narratives. (This, of course, is not to say that you should not seek to master calculus at some point.)
Our focus is on the *big picture ideas* and we use interactive graphics (powered by a widgets module) to help us explore these ideas.

## Learning Objectives for Part (a)

- Understand what a root-finding problem means and how they arise in practice.


- Apply different types of root-finding algorithms to various functions written as `lambda` Python functions.


- Create code that implements root-finding algorithms of different types.


- Create annotations and interactive widgets to enhance visualizations of data.


### Some larger "in situ" learning objectives (i.e., learning that will occur by design of activities)

While we are going to explore how to implement some of these algorithms as activities below, our learning objectives go beyond simple correct implementation. We will also consider what it means to do the following:

- *compare* and *analyze* different algorithms developed for solving the same generic problem;

- use this comparison to *choose* the "right" algorithm for solving a *specific* problem;

- create a module (in the external activity) that encodes various algorithms and a *wrapper* function that automatically chooses which algorithm to apply based on the inputs.

Some of this is done in the notebook while other parts are left for homework.

## Notebook contents <a id='Contents'></a>

* <a href='#Root-finding'>Part (a): Root-finding</a>

    * <a href='#Introduction'>Part (a)(i): An introduction to concepts</a>

    * <a href='#Bracketing'>Part (a)(ii): Bracketing methods</a>

        * <a href='#activity-bisection'>Activity: Implementing the bisection algorithm</a>

        * <a href='#activity-bisection-variant'>Activity: Implementing a bisection variant</a>

        * <a href='#activity-bracketing-compare'>Activity: Comparing bracketing methods</a>

    * <a href='#Iterative'> Part (a)(iii): "Iterative" methods </a>

        * <a href='#activity-secant'>Activity: Implementing the secant algorithm</a>

        * <a href='activity-linear-methods-compare'>Activity: Comparing secant to false point</a>

    * <a href='#activity-summary'>Activity: Summary</a>

## Part (a): Root-finding <a id='Root-finding'></a>
---

**Expected time to completion: 6-9 hours**
    

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('XVh2KPh_Uw4', width=800, height=450)

## Part (a)(i): An introduction to concepts <a id='Introduction'>

### What does root-finding mean?

Simply put, given $f(x)$, a root of the function is some point $x=c$ such that $f(c)=0$. 

In other words, the ***roots*** of a function are the *inputs* that make the *output* equal to zero. 

### lambda functions for simple examples

We only mentioned these briefly in 03-Lecture-part-b.ipynb in the context of list comprehensions, so you may want to review that briefly for some more context.

A lambda function is very useful in the context of this notebook, so we dive into them a bit more here. 

What is a `lambda` function? You can read quite a bit about such functions [here](https://www.w3schools.com/python/python_lambda.asp) or [here](https://realpython.com/python-lambda/).

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- The main takeaway is that if you have a simple type of "anonymous" function that you just need for a short period of time in your code, then a lambda function is probably right for you. It is *not* necessary to use these, but they are useful. 

- A generic use syntax we care about would look something like this:

  > ```f = lambda arguments : expression ```

Run the code cells below to see an example of a `lambda` function in the context of plotting the roots of a polynomial.

Pay attention to how we use these types of functions in this lecture. At times, we treat these as completely anonymous functions in the sense that we pass them directly into algorithms without formally defining them anywhere else in the code. This allows us to study how the algorithms perform on different types of functions rapidly without having to formally declare/create the functions elsewhere in the code.
In other words, they are simply a matter of convenience.

In [ ]:
f = lambda x : x**3 - x - 2  # defining f(x) = x^3-x-2

In [ ]:
# A crash-course in using plots to tell a story in two easy steps
%matplotlib widget

#####################################
# Step 1: Plot your stuff
#####################################
plt.figure(figsize=(10,5), num=0)

x = np.linspace(-1, 2, 100)  # Create 100 uniformly spaced points between -1 and 2
plt.plot(x, f(x), 'b', linewidth=2)  # plot of function f vs x

# Let's add some axes to the plot
plt.axvline(0, linewidth=1, linestyle=':', c='k')  # plot typical y-axis
plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

# Let's add the root to the plot using a scatter plot
c = 1.521  # approximate root
plt.scatter(c, f(c), s=70, c='r', marker='s')  # plot approximate root

######################################
# Step 2: Annotate and add informational text to explain your stuff
######################################
# Create a text string for the title that formats certain floating point numbers
# that are relevant in our plot
title_str = 'Approx. root at c=' + '{:.3}'.format(c) + r' with f(c)$\approx$' + '{:.2e}'.format(f(c))
plt.title(title_str, fontsize=14)

# Now we add fancier annotations to the plot with text and arrows in a two-step process

# First, we define the dict (dictionary) of properties of the text and arrows used in the annotation
bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.5, linewidth=2)  # properties for bounding box of text
arrowprops = dict(arrowstyle='->', color='k', alpha=0.5, linewidth=2)  # properties for the arrow

# Now annotate the plot
plt.annotate('Approx. root', fontsize=12, xy=(c,f(c)), xytext=(c-1.25,1), color='w',
             bbox = bbox, arrowprops = arrowprops)

plt.show();

### A plotting tool class for roots
---

For lots of different functions $f$, we may wish to repeat the above, but we certainly don't want to write out all of this code again. 

Also, the above code had some "hard-coded" elements to it that were based on our prior knowledge about where a root of $ x^3-x-2$ is located. 

In more general cases, we may want a plotting tool associated with a function that allows us to quickly explore where a potential root may be.

We *could* just "functionalize" the code above. However, we are interested in pursuing root-finding problems in this notebook where a particular function does not exist in isolation from specific types of data like its roots. When various "types of things" are intended to be used in an integrated manner, this is a good indication that we should at least consider the use of classes to organize these various "things" as data or method attributes of an object.

Thus, we create a class of plotting tools associated with a function that is useful for this lecture notebook, which also gives us some more experience working with classes.

In the code below, note how we have created a class with fairly short/modularized method attributes that each serve a specific purpose in relationship to the current state of knowledge we have regarding the root of a function we are interested in studying.

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 
- Add more information to the docstrings.

- Provide more useful code comments wherever they are missing.

In [ ]:
class PlotToolsForRoots(object):
    '''
    A class that provides useful plots/information involving the root of a function.
    '''
    def __init__(self, f, root=None):
        '''
        If a root is given, then it should be given as a float.
        '''
        self.f = f
        self.root = root  # May be None type if no root is known at time of initialization

        return
    
    def set_root(self, root):
        '''
        Used to set the self.root attribute as a new root value. Give as a float.
        '''
        self.root = root
        
        return
        
    def plot(self, x_min, x_max, fignum=0, N=100, c='b', ls='-', lw=2, show_root = True, annotate=True, clf = True):
        '''
        Used to visualize the function and potentially its root.
        '''
        self.fig = plt.figure(fignum)
        if clf is True:
            self.fig.clf()
        
        # Let's add x-axis to the plot 
        # We are not adding the y-axis b/c the root may be far away from x=0 which is where typical y-axis is plotted
        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
        
        x_plot = np.linspace(x_min, x_max, N)
        
        plt.plot(x_plot, self.f(x_plot), c=c, ls=ls, lw=lw)
        
        if self.root is not None and show_root:
            self.plot_root(annotate)
        
        plt.show();
        return
        
    def plot_root(self, annotate):
        '''
        Used to plot the root if it has been found/estimated
        '''
        
        plt.scatter(self.root, self.f(self.root), s=70, c='r', marker='s')  # plot approximate root

        # Create a text string for the title that formats certain floating point numbers
        # that are relevant in our plot
        title_str = 'Approx. root at c=' + '{:.3}'.format(self.root) + r' with f(c)$\approx$' + '{:.2e}'.format(self.f(self.root))
        plt.title(title_str, fontsize=14)
        
        if annotate:
            self.plot_annotate()
            
        return
    
    def plot_annotate(self):    
        '''
        Used to annotate the root plot with text and arrows in a two-step process
        '''

        # First, we define the dict (dictionary) of properties of the text and arrows used in the annotation
        bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.5, linewidth=2)  # properties for bounding box of text
        arrowprops = dict(arrowstyle='->', color='k', alpha=0.5, linewidth=2)  # properties for the arrow

        # Now annotate the plot
        y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
        y_range = y_max - y_min
        x_min, x_max = plt.xlim()
        x_range = x_max - x_min

        plt.annotate('Approx. root', fontsize=12, xy=(self.root,self.f(self.root)), 
                     xytext=(self.root-0.35*x_range,0.1*y_range), 
                     color='w', bbox = bbox, arrowprops = arrowprops);
            
        return

In [ ]:
fplot = PlotToolsForRoots(f)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=1)

In [ ]:
fplot.set_root(1.521)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2, annotate=False)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2, show_root=False)

In [ ]:
# Now we see an example that really illustrates why we do not want to plot the typical y-axis

fplot_2 = PlotToolsForRoots(lambda x : (x-2000)*(x-5000)*(x-10000))

In [ ]:
# Pay attention to the y-axis range, it is in units of 1e11 = 10,000,000,000.
# The "largest" value of the function here is about -10,000,000,000.
# In other words, for x between -10 and 10, the function never gets closer 
# than a distance of about 10 billion units from the x-axis.
# We are pretty far off from a root then.

%matplotlib widget
fplot_2.plot(-10, 10, fignum=3)  

In [ ]:
# A plot of x between 1000 and 110000 shows where all three roots are located and
# we also get the sense that the function is just a typical cubic polynomial.
# We do not require the typical y-axis here (which is plotted at x=0). We really
# only need to use the typical x-axis (i.e., y=0) in our plots to see where the function may have roots.

%matplotlib widget
fplot_2.plot(1000, 11000, fignum=4) 

In [ ]:
# Here, we do have knowledge of the exact location of a root at 2000

fplot_2.set_root(2000.)  # Try using int 2000 instead of the float 2000., can you explain the error?

%matplotlib widget
fplot_2.plot(1000, 3000, fignum=5)

***The above examples provide us more of the idea of the "what" but not the "why."***

### Why should we care about roots?

Is there anything particularly important about a function being zero? In general, the answer is no. However, it depends on the function and the goals of the problem/application for which the function models system behavior. 
The assignment discusses how root problems arise naturally in many optimization problems.
Below, we provide some simpler motivating examples.

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Some motivating examples:*** <span>

- Suppose there are two launched projectiles with two height functions denoted by $h_1(t)$ and $h_2(t)$, and we are very interested when, or even *if*, there is a time where $h_1(t)=h_2(t)$. In other words, is there a time where the objects collide? This is an important question that arises in missile defense systems, satellite monitoring, and autonomous vehicle guidance systems. If we define $f(t)=h_1(t)-h_2(t)$, then we are interested in knowing whether or not $f(t)=0$. 

- Suppose $p(t)$ describes a model you develop for the price of a potential investment (it could be in a savings/money market account, stock, or mutual fund to name just a few). You are probably interested in knowing when your model predicts the price to reach a certain target value, $p_{target}$, so that you can develop certain financial plans/goals around this time frame (e.g., having enough for a down payment on a car or house, take a vacation, or retire). Defining $f(t)=p(t)-p_{target}$ means you are interested in determining when $f(t)=0$.

### And these root problems are solved how?

There are lots of algorithms that attempt to approximate the roots of a function, e.g., see https://en.wikipedia.org/wiki/Root-finding_algorithms. 

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- Most algorithms are *iterative*. (Meaning we need to write *loops* to implement them.)

- Some algorithms are very easy to implement. Some are more difficult. Generally, the easier something is to implement, the more restrictive the conditions are under which we expect it to produce anything meaningful. 

## Part (a)(ii): Bracketing methods <a id='Bracketing'>
---

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the second recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('MOk186WuqpE', width=800, height=450)

## Bisection algorithm (https://en.wikipedia.org/wiki/Bisection_method)

---

The basic idea is best illustrated by an interactive demo and playing the video above.

***Recall that we use widgets to create the interactivity. Widgets are very useful for illustrating concepts and exploring ideas.***

In [ ]:
# Usually place these at the top of a notebook if you know you are going to use widgets
# This cell only needs to be executed once
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 
If you really want to understand things, do the following:
- Update the docstring in `compute_bisection` to describe the (1) role of the various parameters, and (2) what variables (if any) the function returns.

- Add at least one doctest.

In [ ]:
def compute_bisection(f, a, b, n=10, tol_interval=1e-5, tol_f=1e-8):
    '''
    This simple function applies up to n iterations of the 
    bisection algorithm
    '''
    
    # Let us first check the conditions to apply the bisection
    # algorithm are satisfied
    if a >= b:
        raise ValueError('The bisec. alg. requires a<b')
    if f(a)*f(b) > 0:
        raise ValueError('The bisec. alg. requires f(a) and f(b) ' +\
                         'are different signs')
        
    # Now, perform the bisection algorithm
    current_iter = 0
    while current_iter < n:
        current_iter += 1
        
        # bisect the interval [a,b], i.e., compute mid-point
        c = (b+a)/2 
        
        # Check if c is an (approx.) root
        if np.abs(f(c)) < tol_f:
            break
        
        # Determine if a or b should be replaced with c
        if f(a)*f(c) < 0:
            b = c
        else:
            a = c
            
        # Check if dividing [a,b] in half made the interval 
        # too small to continue
        if (b-a) < tol_interval:
            break
        
    return (a, b, c)  # Returns a tuple

In [ ]:
# This uses the lambda function f previously defined
# as well as default values for three of the parameters
a, b, c = compute_bisection(f, 0, 2)

# Print the approximate root
print(c)

In [ ]:
# We can also return the tuple of outputs as a single variable
bisec_results = compute_bisection(f, 0, 2)

# The output is a tuple
print(type(bisec_results))

# The length of the tuple
print(len(bisec_results))

# The last component of the tuple has the approx. root
print(bisec_results[-1]) 

In [ ]:
# We could just ignore all the c value returned by the function as follows
_, _, c = compute_bisection(f, 0, 2)

print(c)

In [ ]:
# OR, we could just return only the last value of the tuple
c = compute_bisection(f, 0, 2)[2]

print(c)

In [ ]:
# If we did not know how many components were in the tuple, 
# BUT we did know that the last one had the value we were after, 
# then we could use the following line of code
c = compute_bisection(f, 0, 2)[-1]

print(c)

**Now let's add some interactivity.** Watch what if you choose `b` (or `a`) such that `f(b)` and `f(a)` have the same sign. But, how do we know when that will happen? (*Hint*: refer to the plot.)

In [ ]:
interact(compute_bisection, 
         f = fixed(f),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

**We can easily play with other lambda functions.**

***But,*** what is going on with all of the errors as we try different values of `b` in the code below for the lambda function used?

In [ ]:
interact(compute_bisection, 
         f = fixed(lambda x: np.sin(x**3 - x - 2)),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

**Let's plot the lambda function from the previous code cell to see what is going on.**

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 
- After looking at the plots below (or after making your own), enter in more reasonable values for `a` and `b` in the widget above to see the different roots we could approximate *as long as we use suitable initial values for `a` and `b`*.

In [ ]:
fplot_new = PlotToolsForRoots(lambda x: np.sin(x**3 - x - 2))

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=0, x_max=2)  # Observe that f(x_min) is same sign as f(x_max)

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=1.25, x_max=1.75)  # We could try to find the root that is between 1.25 and 1.75

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=1.75, x_max=2)  # We could also try to find the root that is between 1.75 and 2

**Let's plot a different function that never crosses the x-axis.**

In [ ]:
f_new = lambda x: x**4 - 2*x**3 + x + 1

fplot_new = PlotToolsForRoots(f_new)

In [ ]:
%matplotlib widget

fplot_new.plot(x_min=-2, x_max=3)

**We HOPE to get (in fact we *better get*) an error when we try to find a root to the function above.**

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** </span>

- The `compute_bisection` **is working PROPERLY if it gives us an error in this case.**

- In general, an error produced by a code does not mean that there is an error *in* the code. Many times, the code is operating *as expected* or at least *as intended*. 
    
  - Many errors are *user errors* where the user tries to use the code in a way it is not intended to be used, which causes an *unexpected error*. 

  - When an error is returned by the code that it is written to catch, such as in the `compute_bisection` algorithm, then this is an *expected error* that the programmer knows (either through domain-specific knowledge or by testing code with users and gathering feedback) may happen. 

In [ ]:
interact(compute_bisection, 
         f = fixed(f_new),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

***What is the bigger lesson here?*** 

Plots are *really* useful and we should often do some initial "grunt" work on examining the problem we are asked to solve to make sure that we actually set it up correctly. 

The `compute_bisection` was *not* failing just because it output errors. It was functioning just as it should. We gave it *bad* choices of `a` and `b` as the default values in the widget. Simply looking at the plot reveals this to us right away. 

While we may not always be able to plot a function for many reasons (e.g., it takes too long to evaluate the hundreds to thousands of times necessary to make a reasonable looking plot, the dimension of the input space is simply too high to properly visualize, etc.), **we should always try to visualize the data** associated with a problem *somehow* and in *someway* to orient ourselves properly on the problem.

### A `Bisection` class
---

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the third recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('WS2YLD_aQ7s', width=800, height=450)

With the above lesson in mind along with the broader perspective on the class of problems we are trying to solve, we now create a new class `Bisection` that starts to put all the useful attributes of a root-finding problem together based on the bisection algorithm. 

What should the attributes be?

- **Data attributes:**

  - The function `f`, the `a` and `b` parameters defining the interval $[a,b]$ where we look for a root, and `n` the number of iterations of the algorithm we will make are all obvious data attributes for this class. Since we may not know the `a`, `b`, and `n` we want to use for a particular `f`, we should default these to `None` upon initialization and then have a method attribute that can set these values similar to what we did for the `root` parameter with the method `set_root` in the `PlotToolsForRoots` class above.

  - One of the data attributes should be an instantiation of the `PlotToolsForRoots`. This data attribute, which is made from a class, is quite useful for us in determining the `a` and `b` parameters (and even the `n` parameter).
  
  - The root computed by the bisection algorithm is another data attribute, but it is not determined at initialization. It is determined and set when the bisection algorithm executes.
  
  - The tolerance parameters for the bisection algorithm are other data attributes, and these seem like the types of attributes that should have some default values based on "typical" use cases from prior experience (i.e., we default to values that "feel" right).

- **Method attributes:**

  - The compute bisection algorithm.
  
  - A method for setting the `a`, `b`, and `n` parameters to other values we determine are "better" than what was assigned (if anything) upon initialization. Such values may be determined by examining a plot using the data attribute associated with the `PlotToolsForRoots` class.

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 

- Add docstrings and code comments below.

In [ ]:
class Bisection(object):
    
    def __init__(self, f, a=None, b=None, n=None, tol_interval=1e-5, tol_f=1e-8):
        
        self.f = f
        self.a = a
        self.b = b
        self.n = n
        self.tol_interval = tol_interval
        self.tol_f = tol_f
        self.plot_tool = PlotToolsForRoots(f)
        
        return
        
    def set_bisection_parameters(self, a=None, b=None, n=None, tol_interval=None, tol_f=None):
        
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b
        if n is not None:
            self.n = n
        if tol_interval is not None:
            self.tol_interval = tol_interval
        if tol_f is not None:
            self.tol_f = tol_f
            
        return
            
    def compute_bisection(self):
        '''
        This simple function applies up to n iterations of the 
        bisection algorithm. 
        
        It sets new data attributes based on the outputs of the previously 
        defined compute_bisection standalone function.
        '''
        
        if (self.a is None) or (self.b is None) or (self.n is None):
            raise AttributeError('Check that all bisection parameters are set. \n' +\
                   'Use set_bisection_parameters to set: a, b, and/or n.')

        # Let us first check the conditions to apply the bisection
        # algorithm are satisfied
        if self.a >= self.b:
            raise ValueError('The bisec. alg. requires a<b')
        if self.f(a)*self.f(b) > 0:
            raise ValueError('The bisec. alg. requires f(a) and f(b)' +\
                             'are different signs')

        a_n = self.a
        b_n = self.b
        # Now, perform the bisection algorithm
        current_iter = 0
        while current_iter < self.n:
            current_iter += 1

            # bisect the interval [a,b], i.e., compute mid-point
            c = (b_n + a_n)/2 

            # Check if c is an (approx.) root
            if np.abs(self.f(c)) < self.tol_f:
                break

            # Determine if a or b should be replaced with c
            if self.f(a_n)*self.f(c) < 0:
                b_n = c
            else:
                a_n = c

            # Check if dividing [a,b] in half made the interval 
            # too small to continue
            if (b_n - a_n) < self.tol_interval:
                break
                
        # New data attributes are set based on the completion of the above algorithm        
        self.a_n = a_n 
        self.b_n = b_n
        self.root = c
        self.plot_tool.set_root(c)  # Useful for plotting!

        return

Let's now explore how easy it is to use this class to study a function and approximate its root.

In [ ]:
f_new = lambda x: x**3-x-2

f_bisect = Bisection(f_new)

In [ ]:
%matplotlib widget

f_bisect.plot_tool.plot(-1, 3)

In [ ]:
f_bisect.compute_bisection()  # <---- this will raise an attribute error, why?

In [ ]:
f_bisect.set_bisection_parameters(a = -1, b = 3, n = 10)

In [ ]:
f_bisect.compute_bisection()

In [ ]:
%matplotlib widget

f_bisect.plot_tool.plot(-1, 3)

### Let's explore how to visualize the entire bisection algorithm

Below, we create a plotting function that *wraps* around the `compute_bisection` algorithm to add some illustration to the actual bisection process. 

**Read the comments in the code cell below.**

Note: We first present this as a standalone function for clarity before we include it in a class.

In [ ]:
# This cell creates a plotting function that we 
# then use widgets to interact with

def plot_bisection(f, a, b, n, x_min, x_max, grid_pts = 100, show_final_interval = False, fignum = 0):
    # Compute approximate root with bisec. alg. 
    (a_n, b_n, c) = compute_bisection(f, a, b, n)
    
    #############################################################
    # I cannot emphasize enough that the whole of the scientific
    # computation for approximating the root of f occurs in the 
    # single line of code above that calls the separate function
    # compute_bisection.
    #############################################################
    
    ###############################################################
    # Everything below is just to make fancy illustrative plots of 
    # the results. Read through it carefully. Add/edit comments
    # as you do to make sure you understand what each line of code
    # is doing, but keep in mind that all the scientific computation
    # was already done above. We often put significant effort into
    # visualizing results so that we can properly "tell the story"
    # of the solution.
    ################################################################
    fig = plt.figure(figsize=(10,5), num=fignum)
    x = np.linspace(x_min, x_max, grid_pts)
    
    # First, plot function and axes
    plt.plot(x, f(x), 'b', linewidth=2)  # plot of function f
    plt.axvline(0, linewidth=1, linestyle=':', c='k')  # plot typical y-axis
    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    # Title information
    title_str = 'Approx. root is ' + '{:.6}'.format(c) + '. Use this to update interval [a,b].'
    plt.title(title_str, fontsize=14)
    
    # Create a dict (dictionary) of properties of items used in annotations
    bbox_pt = dict(boxstyle='round', fc='0.9')
    bbox_text = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2) 
    arrowprops = dict(arrowstyle='->', facecolor='black', alpha=0.5, linewidth=2) 
    y_min, y_max = plt.ylim()  # get min and max y-values from plot to offset annotations
    y_range = y_max - y_min
    x_range = x_max - x_min
    
    # Now plot relevant points and annotate
    
    # First annotate parts of the plot involving the points a and b
    if show_final_interval:
        plt.scatter(a_n,f(a_n),s=70,c='k')
        
        plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(a_n,f(a_n)), xytext=(a_n+0.2*x_range,f(a_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b_n,f(b_n),s=70,c='k')
        
        plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(b_n,f(b_n)), xytext=(b_n+0.2*x_range,f(b_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
    else:
        plt.scatter(a,f(a),s=70,c='k')
        
        plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(a,f(a)), xytext=(a+0.1*x_range,f(a)-0.25*y_range),
                     bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b,f(b),s=70,c='k')
        
        plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(b,f(b)), xytext=(b+0.1*x_range,f(b)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)

    # Now annotate the part of the plot involving the approximate root c    
    plt.scatter(c,f(c),s=70,c='r')
    
    plt.annotate('Approx. root\nfrom ' + str(n) + '\nstep(s) of bisec.\nalgorithm', fontsize=12, xy=(c,f(c)), 
                     xytext=(c-0.2*x_range,f(c)+0.25*y_range), color='w',
                     bbox = bbox_text, arrowprops=arrowprops)
    
    plt.annotate(r'$(c,f(c))$', fontsize=12, xy=(c,f(c)), xytext=(c+0.1*x_range,f(c)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
    plt.show()
    return

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

interact_manual(plot_bisection, 
         f = widgets.fixed(lambda x: x**3-x-2),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         grid_pts = widgets.fixed(100),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(0))

In [ ]:
%reset -f out

%matplotlib widget

interact_manual(plot_bisection, 
         f = widgets.fixed(lambda x: np.sin(x**3-x-2)),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(3),
         grid_pts = widgets.fixed(1000),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(1))

### Expanding on the `Bisection` class (and **not** the `PlotToolsForRoots` class)
---

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the third recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('IqZBPqve-CM', width=800, height=450)

Let's expand the `Bisection` class to include this useful plotting function that visualizes the steps in the bisection algorithm.

Why do we want to put it in the `Bisection` class and not in the `PlotToolsForRoots` class? 

The plotting function we are interested in coding fundamentally relies upon the outputs of the bisection algorithm (it is the first step in the function). Since we may use `PlotToolsForRoots` to help us setup other root-finding algorithms other than the bisection algorithm (in fact, we do just this below), we do not want to clutter this class with specific plotting functions that will not work when used inside of other classes defined by other root-finding methods. 

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 

- Add docstrings and code comments below.

In [ ]:
class Bisection(object):
    
    def __init__(self, f, a=None, b=None, n=None, tol_interval=1e-5, tol_f=1e-8):
        
        self.f = f
        self.a = a
        self.b = b
        self.n = n
        self.tol_interval = tol_interval
        self.tol_f = tol_f
        self.plot_tool = PlotToolsForRoots(f)
        
        return
        
    def set_bisection_parameters(self, a=None, b=None, n=None, tol_interval=None, tol_f=None):
        
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b
        if n is not None:
            self.n = n
        if tol_interval is not None:
            self.tol_interval = tol_interval
        if tol_f is not None:
            self.tol_f = tol_f
            
        return
            
    def compute_bisection(self):
        '''
        This simple function applies up to n iterations of the 
        bisection algorithm. 
        
        It sets new data attributes based on the outputs of the previously 
        defined compute_bisection standalone function.
        '''
        
        if (self.a is None) or (self.b is None) or (self.n is None):
            raise AttributeError('Check that all bisection parameters are set. \n' +\
                   'Use set_bisection_parameters to set: a, b, and/or n.')
            return

        # Let us first check the conditions to apply the bisection
        # algorithm are satisfied
        if self.a >= self.b:
            raise ValueError('The bisec. alg. requires a<b')
        if self.f(a)*self.f(b) > 0:
            raise ValueError('The bisec. alg. requires f(a) and f(b)' +\
                             'are different signs')

        a_current = self.a
        b_current = self.b
        # Now, perform the bisection algorithm
        current_iter = 0
        while current_iter < self.n:
            current_iter += 1

            # bisect the interval [a,b], i.e., compute mid-point
            c = (b_current+a_current)/2 

            # Check if c is an (approx.) root
            if np.abs(self.f(c)) < self.tol_f:
                break

            # Determine if a or b should be replaced with c
            if self.f(a)*self.f(c) < 0:
                b_current = c
            else:
                a_current = c

            # Check if dividing [a,b] in half made the interval 
            # too small to continue
            if (b_current-a_current) < self.tol_interval:
                break
                
        # New data attributes are set based on the completion of the above algorithm        
        self.a_n = a_current  
        self.b_n = b_current
        self.root = c
        self.plot_tool.set_root(c)  # Useful for plotting!

        return
    
    def plot_bisection(self, x_min, x_max, a = None, b = None, n = None, grid_pts = 100, 
                       show_final_interval = False, fignum = 0):
        '''
        Plot information related to the performance of the bisection algorithm
        '''
        
        # Update bisection with any new a, b, or n values
        self.set_bisection_parameters(a=a, b=b, n=n)
        
        # Compute approximate root with bisec. alg. 
        self.compute_bisection()

        self.plot_tool.plot(x_min=x_min, x_max=x_max, N=grid_pts, fignum=fignum, show_root=False)
    
        self.plot_bisection_annotate(x_min, x_max, show_final_interval)
        
        return

    def plot_bisection_annotate(self, x_min, x_max, show_final_interval):
        '''
        Annotate the information plotted related to the performance of the bisection algorithm
        '''
        
        # Create a dict (dictionary) of properties of items used in annotations
        bbox_pt = dict(boxstyle='round', fc='0.9')
        bbox_text = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2) 
        arrowprops = dict(arrowstyle='->', facecolor='black', alpha=0.5, linewidth=2) 
        y_min, y_max = plt.ylim()  # get min and max y-values from plot to offset annotations
        y_range = y_max - y_min
        x_range = x_max - x_min

        # Now plot relevant points and annotate

        # First annotate parts of the plot involving the points a and b
        if show_final_interval:
            plt.scatter(self.a_n,self.f(self.a_n),s=70,c='k')
            
            plt.scatter(self.b_n,self.f(self.b_n),s=70,c='k')

            if self.f(self.a) > 0:
                plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(self.a_n, self.f(self.a_n)), 
                         xytext=(self.a_n-0.25*x_range, self.f(self.a_n)-0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(self.b_n, self.f(self.b_n)), 
                         xytext=(self.b_n+0.25*x_range, self.f(self.b_n)+0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
            else:
                plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(self.a_n,self.f(self.a_n)), 
                         xytext=(self.a_n-0.25*x_range, self.f(self.a_n)+0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(self.b_n,self.f(self.b_n)), 
                         xytext=(self.b_n-0.25*x_range, self.f(self.b_n)-0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)

        else:
            plt.scatter(self.a,self.f(self.a),s=70,c='k')
            
            plt.scatter(self.b,self.f(self.b),s=70,c='k')

            if self.f(self.a) > 0:
                plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(self.a,self.f(self.a)), 
                         xytext=(self.a+0.1*x_range, self.f(self.a)-0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(self.b,self.f(self.b)), 
                         xytext=(self.b-0.25*x_range, self.f(self.b)+0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
            else:
                plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(self.a,self.f(self.a)), 
                         xytext=(self.a+0.1*x_range, self.f(self.a)+0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(self.b,self.f(self.b)), 
                         xytext=(self.b-0.25*x_range, self.f(self.b)-0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)

        # Now annotate the part of the plot involving the approximate root   
        plt.scatter(self.root,self.f(self.root),s=70,c='r')

        plt.annotate('Approx. root\nfrom ' + str(self.n) + '\nstep(s) of bisec.\nalgorithm', fontsize=12, 
                     xy=(self.root,self.f(self.root)), 
                     xytext=(self.root-0.25*x_range, self.f(self.root)+0.2*y_range), color='w',
                     bbox = bbox_text, arrowprops=arrowprops)
        
        plt.annotate(r'$(c,f(c))$', fontsize=12, xy=(self.root,self.f(self.root)),
                     xytext=(self.root+0.1*x_range,self.f(self.root)-0.1*y_range),
                     bbox=bbox_pt, arrowprops=arrowprops)
        
        return

In [ ]:
f_new = lambda x: x**3-x-2

f_bisect = Bisection(f_new)

f_bisect.set_bisection_parameters(a = -1, b = 3, n = 10)

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

f_bisect.plot_bisection(-1, 3)

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

%reset -f out

interact_manual(f_bisect.plot_bisection, 
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(3),
         grid_pts = widgets.fixed(1000),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(0))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Playing with the bisection algorithm</span><a id='activity-bisection'></a>
    
**Note 1**: You may want/need to create additional code (and Markdown) cells below as you work your way through this activity.
    
**Note 2**: You will probably need to explore some plots of the functions first (e.g., by experimenting with the `plot_tool` attribute of instantiations of the `Bisection` class)  to determine viable candidates for the values of `a` and `b` in order to capture the root for the functions you are playing with. Suppose the root is $100$, then you need to setup the problem and any associated plots around this value to get things to work properly. 
    
**Note 3**: It is okay to show examples that produce errors. This is in fact *encouraged* as long as you explain the errors. But, you should also show results that do not produce errors.

**In this activity, you should do the following (by the way, "some" always means at least two):**    
    
- Create instantiations of the `Bisection` class for **some** (meaning at least two) `lambda` functions of your choosing that have roots.
<br><br>
  - Demonstrate approximations of these roots for each `lambda` function using the `plot_bisection` method attribute of the associated instantiation of the `Bisection` class. At least one of these plots should be without widgets and at least one of these plots should use widgets for interactivity.
<br><br>    
  - Summarize results in a Markdown cell you create below the plots.

Notice that the method attribute `plot_bisection` does not have `tol_interval` or `tol_f` as parameters. When it calls `compute_bisection`, it is simply relying upon default values for these parameters. We should improve upon that.


- **Copy/paste the `Bisection` class below and rename it `BisectionImproved`.** 
<br><br>
  - Update the `plot_bisection` method attribute that allows for all of the parameters in the `set_bisection_parameters` method to be set by the user.
  
*Hint:* This requires updating just two things. First, the parameters passed to the method `plot_bisection`. Second, the call to `set_bisection_parameters` within this method.



- Redo the first part of this activity utilizing your `BisectionImproved` class for **some** `lambda` functions of your choosing both **with and without widgets** where you show results for different choices of these new tolerance parameters. 
    
*Hint:* **You should use a FloatLogSlider for the tolerance parameters** since we often vary tolerance parameters by orders of magnitude when we are testing out an algorithm.

<hr style="border:5px solid cyan"> </hr>

## A bisection variant (https://en.wikipedia.org/wiki/Regula_falsi)

The basic idea of *regula falsi* (i.e., the false position method) is quite simple. It follows 3 steps starting with an initial guess of an interval $[a,b]$ that may contain a root. 

1. Make a straight line between $(a,f(a))$ and $(b,f(b))$. 

2. Compute the $x$-intercept of this line and call that point $c$. 

3. Compute $f(c)$ and compare the sign to $f(a)$ (or $f(b)$) and update the interval to either $[a,c]$ or $[c,b]$ depending on the result of this comparison.

Thus, this is just a variant of the bisection algorithm. The fundamental difference is that instead of using the midpoint of the interval $[a,b]$ as the guess for the root $c$, we use *interpolation* to approximate the function with a simple function (a line) for which it is easy to determine the root. The exact root of this approximating function is then an approximate root of the exact function.

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Implementing a bisection variant</span><a id='activity-bisection-variant'>

**The same three notes as the previous activity also apply here.**

- Create a class `FalsePosition` that performs the false position method by following the steps below.
    
Step 1: Copy/paste the `Bisection` class (or `BisectionImproved`) below and edit its name to `FalsePosition`.
    
Step 2: Rename method attributes that reference `bisection` with `false_position`.
    
Step 3: In the now named `compute_false_position` method attribute, edit the single line of code in that method involved with computing `c` (the wiki article tells you exactly what the value of `c` should be in the false position algorithm). 
    
Step 4: In the newly named plotting methods, make sure to edit any code lines that are referring to methods from the bisection algorithm.
    
Step 5: Edit some of the code comments and the docstrings so that they are now referencing the false position algorithm instead of the bisection algorithm. You should also update the plotting methods (both their names and how they refer to other methods in this new class).
    
**One more thing: make sure that the new `compute_false_position` method attribute tests that the conditions are appropriate for running this algorithm.** 
    
While not required, it is highly suggested that you also put in useful docstrings and doctests within the class and its method attributes.
    
If you want to get *really* fancy, then try to plot the "last" line used to approximate the root. This is *not* required, but it is also not that difficult. You may want to try that after viewing the secant method plots in the next part of this notebook.


- Redo the first part of the previous activity using `FalsePosition` on the same `lambda` functions as in the previous activity with and without widgets. 

The next activity will explore differences we may expect in root approximations obtained by the `FalsePosition` method compared to the `Bisection` method, so you do not need to dive deeply into that here although you should still comment on any differences in Markdown cells.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Comparing bracketing methods</span><a id='activity-bracketing-compare'>

In computational and data science, we often must *choose* between implementing competing algorithms that solve the same class of problem in different ways. An important lesson is that if two or more algorithms exist to solve a problem, then there will be cases where one is preferable to the other. The question then becomes, which should you apply for a particular problem?   
    
- Use a mixture of markdown and code cells below to compare the bisection algorithm and false position method. 

  You may find the wiki articles useful to reference here as well as doing some searching on Google (or your favorite search engine). 

  Describe and implement problems where one method seemingly does better than another (e.g., what happens if the function is approximately linear near its root? what happens if the function is very "flat" near its root?)? Can you explain why that is happening? When would you consider using one method over another? What questions about these methods and comparing them seem worth exploring to you? This is intentionally left as open as possible for you to pursue what you think are intriguing questions to pose, research, and answer.

<hr style="border:5px solid cyan"> </hr>

## Part (a)(iii) "Iterative" methods <a id='Iterative'>
---
    
<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the fourth recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('O7JBKjVYiaM', width=800, height=450)

Iterative methods in root-finding problems generally refer to approaches that require an *initial guess of the root* instead of an initial interval/set containing the root. 
These methods generally rely on defining an *auxiliary* function (i.e., an approximation to the function) that is constructed at the current guess in order to form the next "best" guess at the root of the true function. 


I imagine the incredulity you must feel. After all, weren't the above methods iterative?
Well, yes they were in a very technical sense that they implement loops. 
In fact, the false-point method somewhat hints at the basic idea of these methods and looks similar to the *secant* method.
The difference is that bracketing methods also require a description of a set that contains the root you are looking for, which is significantly more information to specify at the start of a problem than just a single initial guess at the root.

One of the things you will need to consider, and which the instructions for the previous activity above hints at, is why some methods may be preferred to others in various scenarios. 

## The secant method (https://en.wikipedia.org/wiki/Secant_method)

This is perhaps best explained via an interactive demo and playing the video above. 

The wiki article actually includes Python code on this simple algorithm, which we copy below before the demo.

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Good questions to keep in mind:*** </span>

- Can you see where this code can immediately have some difficulty? 

- What would be a good initial check to ensure that the code does not "crash"? 

    - Should a similar check be done at the end of each iteration? 

In [ ]:
# Copied/edited from the wiki article
def secant_method(f, x0, x1, n):
    '''
    Return the root calculated using the secant method.
    '''
    for i in range(n):
        x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        x0, x1 = x1, x2
    return x2

In [ ]:
# Copied and edited from the wiki article for demo purposes.
# We keep track of the iterative results (approx. roots and
# the function values of these approx. roots) by appending
# to a list so that we can (1) examine all the results, and 
# (2) create some fancy plots.
def secant_method_4_demo(f, x0, x1, n):
    '''
    Return the root calculated using the secant method and a list of 
    results at each iteration.
    '''
    xs = [x0, x1]
    fs = [f(x0), f(x1)]
    for i in range(n):
        x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        x0, x1 = x1, x2
        xs.append(x2)
        fs.append(f(x2))
    return xs, fs

**The most difficult thing to understand in the code below is how the `n` secant lines are plotted. Students should try to comment those lines and make sense of how that is working.**

It is best to do that before we turn all of this into a class.

In [ ]:
def plot_secant(f, x0, x1, n,  x_min, x_max, grid_pts = 100, fignum=0):
    # Perform the secant method
    xs, fs = secant_method_4_demo(f, x0, x1, n)
    
    ############################################
    # Everything below this line involves making plots of the results 
    ############################################
    fig = plt.figure(figsize=(8,5), num=fignum)
    x = np.linspace(x_min,x_max,grid_pts)
    
    # Plot function and axes
    plt.plot(x,f(x),'b',linewidth=2) #plot of function f
    plt.axvline(0, linewidth=1, linestyle=':', c='k') #plot typical y-axis
    plt.axhline(0, linewidth=1, linestyle=':', c='k') #plot typical x-axis
    
    # Title information
    title_str = 'The n=' + str(n) + r' iter. of the sec. mtd. gives $f(x_{n+1})\approx$' + '{:.2e}'.format(fs[-1])
    plt.title(title_str, fontsize=18)
    
    # Plot all the n secant lines
    for i in range(n):
        min_idx = np.argmin(xs[i:i+3])
        max_idx = np.argmax(xs[i:i+3])
        if (min_idx != 2) and (max_idx != 2):
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [fs[i:i+3][min_idx], fs[i:i+3][max_idx]],
                     'k', linewidth=1, alpha=0.5)
        elif min_idx == 2:
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [0, fs[i:i+3][max_idx]],
                     'k', linewidth=1, alpha=0.5)
        else:
            plt.plot([xs[i:i+3][min_idx], xs[i:i+3][max_idx]], 
                     [fs[i:i+3][min_idx], 0],
                     'k', linewidth=1, alpha=0.5)
            
    # Make clear what the final secant line is        
    plt.plot([xs[i], xs[i]], [0, fs[i]], 'r', linewidth=1, alpha=0.5)
    plt.plot([xs[i+1], xs[i+1]], [0, fs[i+1]], 'r', linewidth=1, alpha=0.5)    
    plt.scatter(np.array(xs),0*np.array(xs))
    
    # Annotate relevant points in the plot
    bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2)
    y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
    y_range = y_max - y_min
    for i in range(n+2):
        if fs[i] < 0:
            plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(xs[i],y_range*0.05), color='w', bbox = bbox)
        else:
            plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(xs[i],-y_range*0.05), color='w', bbox = bbox)
    plt.show()

In [ ]:
%reset -f out

%matplotlib widget

interact_manual(plot_secant,
         f = widgets.fixed(lambda x: x**3-x-2),
         x0 = widgets.FloatSlider(value=0.5, min=0, max=2.5, step=0.01),
         x1 = widgets.FloatSlider(value=2, min=0, max=2.5, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=10),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(2.5),
         grid_pts = widgets.fixed(1000),
         fignum=widgets.fixed(0))

### Create a `Secant` class

Notice below that we use `tol_f` as a new parameter not previously considered with the secant method functions above. Pay attention to where this is used in the `compute_secant` method inside the class. This adds just a little bit more sophistication to the algorithm in that we may "break" out of the algorithm earlier if the current approximation to the root produces a function value that is of "sufficiently small magnitude" (as determined by this tolerance parameter).

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 

- Add docstrings and code comments below.

In [ ]:
class Secant(object):
    def __init__(self, f, x0=None, x1=None, n=None, tol_f=1e-8):
        
        self.f = f
        self.x0 = x0
        self.x1 = x1
        self.n = n
        self.tol_f = tol_f
        self.plot_tool = PlotToolsForRoots(f)
        
        return
        
    def set_secant_parameters(self, x0=None, x1=None, n=None, tol_f=None):
        
        if x0 is not None:
            self.x0 = x0
        if x1 is not None:
            self.x1 = x1
        if n is not None:
            self.n = n
        if tol_f is not None:
            self.tol_f = tol_f
            
        return
    
    def compute_secant(self):
        '''
        Return the root calculated using the secant method and a list of 
        results at each iteration.
        '''
        self.xs = [self.x0, self.x1]
        self.fs = [self.f(self.x0), self.f(self.x1)]
        
        for i in range(self.n):
        
            x_temp = self.xs[-1] - self.f(self.xs[-1]) * (self.xs[-1] - self.xs[-2]) / \
                    (self.f(self.xs[-1]) - self.f(self.xs[-2]))
            self.xs.append(x_temp)
            self.fs.append(self.f(self.xs[-1]))
            
            if np.abs(self.fs[-1]) < self.tol_f:
                break
         
        self.total_iter = i+1
        self.root = self.xs[-1]
        self.plot_tool.set_root(self.root)

        return 
    
    def plot_secant(self, x_min, x_max, x0=None, x1=None, n=None, grid_pts = 100, fignum=0):
        
        # Update secant with any new x0, x1, or n values
        self.set_secant_parameters(x0=x0, x1=x1, n=n)
        
        # Compute approximate root with secant algorithm 
        self.compute_secant()

        self.plot_tool.plot(x_min=x_min, x_max=x_max, N=grid_pts, fignum=fignum, show_root=False)
    
        self.plot_secant_annotate(x_min, x_max)
               
    def plot_secant_annotate(self, x_min, x_max):
        
        # Title information
        title_str = 'The n=' + str(self.n) +\
                    r' iter. of the sec. mtd. gives $f(x_{n+1})\approx$' +\
                    '{:.2e}'.format(self.fs[-1])
        plt.title(title_str, fontsize=18)

        # Plot all the n secant lines
        for i in range(self.total_iter):
            min_idx = np.argmin(self.xs[i:i+3])
            max_idx = np.argmax(self.xs[i:i+3])
            if (min_idx != 2) and (max_idx != 2):
                plt.plot([self.xs[i:i+3][min_idx], self.xs[i:i+3][max_idx]], 
                         [self.fs[i:i+3][min_idx], self.fs[i:i+3][max_idx]],
                         'k', linewidth=1, alpha=0.5)
            elif min_idx == 2:
                plt.plot([self.xs[i:i+3][min_idx], self.xs[i:i+3][max_idx]], 
                         [0, self.fs[i:i+3][max_idx]],
                         'k', linewidth=1, alpha=0.5)
            else:
                plt.plot([self.xs[i:i+3][min_idx], self.xs[i:i+3][max_idx]], 
                         [self.fs[i:i+3][min_idx], 0],
                         'k', linewidth=1, alpha=0.5)

        # Make clear what the final secant line is        
        plt.plot([self.xs[i], self.xs[i]], [0, self.fs[i]], 'r', linewidth=1, alpha=0.5)
        plt.plot([self.xs[i+1], self.xs[i+1]], [0, self.fs[i+1]], 'r', linewidth=1, alpha=0.5)    
        plt.scatter(np.array(self.xs),0*np.array(self.xs))

        # Annotate relevant points in the plot
        bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2)
        y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
        y_range = y_max - y_min
        for i in range(self.total_iter+2):
            if self.fs[i] < 0:
                plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(self.xs[i],y_range*0.05), color='w', bbox = bbox)
            else:
                plt.annotate(r'$x_' + str(i) + '$', fontsize=12, xy=(self.xs[i],-y_range*0.05), color='w', bbox = bbox)
                
        return

In [ ]:
f_new = lambda x: x**3-x-2

f_secant = Secant(f_new)

f_secant.set_secant_parameters(x0 = 0.5, x1 = 2, n = 10)

In [ ]:
%matplotlib widget

f_secant.plot_secant(-2, 3, n=3)

In [ ]:
%reset -f out

%matplotlib widget

interact_manual(f_secant.plot_secant,
         x0 = widgets.FloatSlider(value=0.5, min=0, max=2.5, step=0.01),
         x1 = widgets.FloatSlider(value=2, min=0, max=2.5, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=10),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(2.5),
         grid_pts = widgets.fixed(1000),
         fignum=widgets.fixed(0))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Improving the secant algorithm</span><a id='activity-secant'>

The same notes as the first two activities apply here.

- Create a `SecantImproved` class that is *improved* from what is implemented in `Secant` in the sense that it checks for and avoids any potential errors (e.g., division by zero?) that may occur both before any computation takes place and also during runtime. Also, allow the `plot_secant` method in the class to set the `tol_f` parameter.

- Test your code on some `lambda` functions and using the interact widget on the plotting method for each instantiation of the class.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Comparing secant to false position</span><a id='activity-linear-methods-compare'>

- Use a mixture of code and markdown cells to compare the secant and false position methods across a variety of `lambda` functions. 

  You may find the wiki articles useful to reference here as well as doing some searching on Google or your favorite search engine. 

  Can you describe and implement problems where one method seemingly does better than another? Can you explain why that is happening? When would you consider using one method over another? What questions about these methods and comparing them seem worth exploring to you? This is intentionally left as open as possible for you to pursue what you think are intriguing questions to pose, research, and answer.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Summary</span> <a id='activity-summary'/>

Summarize some of the key takeaways/points from this notebook in a list below and prepare a few code examples related to these takeaways/points in the code cells below. You need to have at least one example for each of your summary points and you need at least three summary points.

In this notebook, we have seen the following:

- [Your summary point 1 goes here]




- [Your summary point 2 goes here]




- [Your summary point 3 goes here]

<hr style="border:5px solid cyan"> </hr>

### <a href='#Contents'>Click here to return to Notebook Contents</a>